In [1]:
import pandas as pd
pd.__version__

u'0.22.0'

In [0]:
#!pip install spacy

In [0]:
import numpy as np
import io
from nltk import ngrams


In [0]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

#2. Get the file
downloaded = drive.CreateFile({'id':'1CUaLwXYDAjmIZsvS27bhbp0N0kwt4_5y'}) # replace the id with id of file you want to access
downloaded.GetContentFile('train.csv')  


In [5]:
#3. Read file as panda dataframe
df = pd.read_csv('train.csv') 

/usr/local/lib/python2.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [0]:
df2 = df.copy()


In [7]:
df2 = df2.dropna()

df2['label'] = df2['label'].apply(lambda x : int(x))
df2['text'] = df2['text'].apply(lambda x : x.lower().rstrip().replace('\n', ' ').replace('\\\'', '\''))

"""train_frac = 0.8
train = df2.sample(frac = train_frac, random_state = 4269)
test = df2.drop(train.index)

train = train.reset_index()
test = test.reset_index()"""




'train_frac = 0.8\ntrain = df2.sample(frac = train_frac, random_state = 4269)\ntest = df2.drop(train.index)\n\ntrain = train.reset_index()\ntest = test.reset_index()'

In [8]:
df2 = df2.sample(frac=1).reset_index(drop=True)
train_ind = int(0.8*len(df2))
train_ind


320000

In [0]:
#train['one'] =  train['text'].apply(lambda x : list(ngrams(x.split(), 1)))

In [0]:
#train['two'] =  train['text'].apply(lambda x : list(ngrams(x.split(), 2)))


In [0]:
#train['three'] =  train['text'].apply(lambda x : list(ngrams(x.split(), 3)))


In [0]:
corpus = []
#X = []
Y = []
exclaim = []
#question = []
for i in xrange(len(df2)):
  #X.insert(i, train['one'][i] + train['two'][i])
  Y.insert(i, [df2['label'][i]])
  corpus.insert(i, df2['text'][i])
  exclaim.insert(i, [1 if df2['text'][i].find('!') else 0])
  #question.insert(i, [1 if df2['text'][i].find('?') else 0])

#X = np.array(X)
#Y = np.array(Y)

In [11]:
for i in xrange(5):
  print corpus[i]
  print Y[i]

after struggling for years with maintaining my composure in competitive golf events, i finally decided to do something about it.  luckily, i found dr. kev.  in just a few sessions of working with dr. kev, with committed focus on my part, i was able to understand where i was challenged most and apply the lessons dr. kev taught me. as a result, i won my first state tournament!  also, my partner and i were the medalists at the qualifier for the inaugural usga women's four ball.  dr. kev has completely changed my approach to tournament golf.  i simply would not be the player and competitor i am today without his guidance.  most players will take swing lessons and beat balls on the range but pay little to no attention to the mental aspects of the game.  if you really want to improve and achieve your best, i strongly recommend working with dr. kev.
[2]
this place is really nice and very very very clean (upscale). i think they do an amazing job, i like it here and would definitely recommend i

In [12]:
from sklearn.feature_extraction.text import CountVectorizer
#import nltk
#nltk.download('stopwords') 
vectorizer = CountVectorizer(ngram_range=(1, 2), stop_words=['the', 'a'])
#, stop_words=set(nltk.corpus.stopwords.words('english')))
vectorizer

CountVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 2), preprocessor=None, stop_words=['the', 'a'],
        strip_accents=None, token_pattern=u'(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [13]:
vect3 = vectorizer.fit_transform(corpus)
vect3

<400000x4962217 sparse matrix of type '<type 'numpy.int64'>'
	with 64538441 stored elements in Compressed Sparse Row format>

In [14]:
from scipy.sparse import hstack, csr_matrix
vect2 = hstack([vect3, exclaim])
#vect = hstack([vect2, question])
vect = csr_matrix(vect2)
vect

<400000x4962218 sparse matrix of type '<type 'numpy.int64'>'
	with 64938430 stored elements in Compressed Sparse Row format>

In [15]:
from sklearn import svm

lin_clf = svm.LinearSVC(class_weight='balanced', multi_class='ovr', max_iter=2000, )
lin_clf.fit(vect[:train_ind], np.ravel(Y[:train_ind]))

LinearSVC(C=1.0, class_weight='balanced', dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=2000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [16]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

y_pred = lin_clf.predict(vect[train_ind:])
y_test = Y[train_ind:]
print(f1_score(y_test, y_pred, average="macro"))
print(precision_score(y_test, y_pred, average="macro"))
print(recall_score(y_test, y_pred, average="macro"))    

0.864272773411421
0.8712123340713097
0.8581974905236588


In [17]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[14623,  1181,   493],
       [ 1349,  9677,  2480],
       [  361,  1606, 48230]])

In [0]:
downloaded = drive.CreateFile({'id':'1-nARzPoOjxSAFcbvv_eMeWvNs86ZaeUI'}) # replace the id with id of file you want to access
downloaded.GetContentFile('test_set.csv')  
df_ftest = pd.read_csv('test_set.csv') 

In [22]:
vectorizer.vocabulary_

{u'realy soon': 3511569,
 u'rightnow explained': 3630895,
 u'shebangabang after': 3846016,
 u'clergy they': 924067,
 u'fare haven': 1569015,
 u'meetings we': 2689531,
 u'hardware isn': 1983424,
 u'circus skills': 905277,
 u'pressure so': 3379651,
 u'to saleswoman': 4406238,
 u'track understand': 4458914,
 u'laughing girl': 2435680,
 u'hop we': 2099091,
 u'her forgetting': 2043261,
 u'meals alright': 2673810,
 u'maybe poor': 2658351,
 u'sushi seems': 4168700,
 u'fart noise': 1570556,
 u'goods hanging': 1873217,
 u'system wii': 4183406,
 u'was lush': 4710983,
 u'recumbent bike': 3535527,
 u'good groceries': 1867696,
 u'vide frenched': 4647267,
 u'about timeshares': 91679,
 u'bit clattery': 588971,
 u'point rd': 3327675,
 u'point re': 3327676,
 u'personal highlights': 3251855,
 u'way freshly': 4734581,
 u'riches he': 3623322,
 u'sgv san': 3830905,
 u'plate really': 3308315,
 u'fucks me': 1767585,
 u'strip give': 4110622,
 u'ever entire': 1486439,
 u'popeyes location': 3340640,
 u'for lug'

In [0]:
vectorizer2 = CountVectorizer(ngram_range=(1, 2), stop_words=['the', 'a'], vocabulary=vectorizer.vocabulary_)


In [0]:

df_ftest['text'] = df_ftest['text'].apply(lambda x : x.lower().rstrip().replace('\n', ' ').replace('\\\'', '\''))

In [24]:
corpus_t = []
#X = []
exclaim_t = []
#question = []
for i in xrange(len(df_ftest)):
  #X.insert(i, train['one'][i] + train['two'][i])
  corpus_t.insert(i, df_ftest['text'][i])
  exclaim_t.insert(i, [1 if df_ftest['text'][i].find('!') else 0])
  
len(df_ftest)

100000

In [25]:
vect_t = vectorizer2.fit_transform(corpus_t)
vect_t

<100000x4962217 sparse matrix of type '<type 'numpy.int64'>'
	with 15344948 stored elements in Compressed Sparse Row format>

In [26]:
vect_t2 = hstack([vect_t, exclaim_t])
#vect = hstack([vect2, question])
vect_t2 = csr_matrix(vect_t2)
vect_t2

<100000x4962218 sparse matrix of type '<type 'numpy.int64'>'
	with 15444940 stored elements in Compressed Sparse Row format>

In [0]:
y_t_pred = lin_clf.predict(vect_t2)


In [47]:
df_ret = pd.DataFrame(y_t_pred)
df_ret.index += 1 
df_ret['ID'] = df_ret.index
df_ret.columns = ['label', 'ID']
df_ret = df_ret[['ID', 'label']]
df_ret

,ID,label
1,1,2
2,2,2
3,3,2
4,4,0
5,5,0
6,6,2
7,7,2
8,8,2
9,9,0
10,10,2


In [0]:
df_ret.to_csv("result_svm.csv", index=False)

In [0]:
from google.colab import files
files.download('result_svm.csv')